## BlackCat Organizations
5/19
* Most recent records
* Transit agencies that have fixed-route services (more than the vendors contracted to  provide dial-a-ride / paratransit services—I recall there being a bunch of those in the  BlackCat export I got last year).
* 5311, SGR then + Clovis

In [1]:
import fuzzywuzzy
import _utils
import pandas as pd
import siuba  # need this to do type hint in functions
from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis.tables import tbls
from fuzzywuzzy import process
from siuba import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/cloud/bigquery/table.py:43: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Merge BlackCat w/ SGR

In [3]:
def blackcat_sgr_service_type_final():
    
    grants_subset = [
    "5311(f) Cont",
    "CMAQ (FTA 5311)",
    "Section 5311",
    "5311(f) Round 2",
    "5339 (State)",
    "Section 5311(f)"]
    
    # Open up the files
    blackcat = _utils.blackcat_orgs(file_name = "BlackCat_Grants_Projects_5_22_23.xlsx",
                             year_wanted = 2018,
                             grants_wanted = grants_subset)
    
    sgr = _utils.sgr_orgs(file_name = "SGR Calsmart-user-list request.xls")
    
    # Use fuzzywuzzy package to change the agency names
    # in Blackcat to match names in SGR before merging
    # if rerunning this in the future, look at results from
    # fuzzy_match_sgr_bc() to see what else to delete
    fuzzy_to_del = [
    "Amador Transit",
    "Eastern Contra Costa Transit Authority",
    "Madera County"]
    
    blackcat = _utils.fuzzy_match_sgr_bc(blackcat, sgr, fuzzy_to_del)
    blackcat_sgr = _utils.merge_blackcat_sgr(blackcat, sgr)
    
    # Use fuzzywuzzy package to change agency names
    # from Blackcat and SGR to names in airtable
    # before merging.
    matches_to_del = [
    "Eastern Contra Costa Transit Authority",
    "Fresno County",
    "Eastern Sierra Transit Authority",
    "Alpine County Local Transportation Commission"
    "Livermore Amador Valley Transit Authority",
    "Calaveras Transit",
    "City Of Corcoran - Corcoran Area Transit",]
 
    blackcat_sgr_airtable = _utils.merge_blackcat_sgr_airtable(blackcat_sgr, matches_to_del)
    
    return blackcat_sgr_airtable

In [4]:
final = blackcat_sgr_service_type_final()

/home/jovyan/data-analyses/grant_misc/_utils.py:60: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[agency_col]
/home/jovyan/data-analyses/grant_misc/_utils.py:96: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  to_correct[["name_pt1", "name_pt2"]] = to_correct[agency_col].str.split(
/home/jovyan/data-analyses/grant_misc/_utils.py:60: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[agency_col]
/home/jovyan/data-analyses/grant_misc/_utils.py:96: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  to_correc

In [8]:
final.sample(3)

,funding_program,organization_name,first_name,last_name,email,phone,title,service_type
180,"Section 5311, State of Good Repair",Santa Clara Valley Transportation Authority,Lorena,Bernal-Vidal,lorena.bernal-vidal@vta.org,4089524235.00,Senior Transportation Planner,Fixed-Route
21,State of Good Repair,City Of California City,Joe,Barragan,jbarragan@californiacity-ca.gov,7603737162.00,Public Works Director,No Service Info
118,State of Good Repair,Lake Countycity Area Planning Council,James,Sookne,apedrotti@dbcteam.net,7072343314.00,Project Manager,No Service Info


In [5]:
# m3.to_excel(f"{gcs_path}5311_SGR_Recipients.xlsx")